In [37]:
from pathlib import Path
from datetime import timedelta
from utinni import Context
from tiro.core import Scenario
from tiro.plugins.utinni import TiroTSPump
import yaml

from rich import print

In [2]:
scenario = Scenario.from_yaml(
    Path("scenario/scenario.yaml"),
    Path("scenario/use-srv1.yaml"),
    Path("scenario/use-srv2.yaml"),
)
pump = TiroTSPump(
    scenario=scenario,
    influxdb_url="http://localhost:8086",
    influxdb_token="tiro-token",
    influxdb_org="tiro",
    arangodb_db="tiro",
    arangodb_graph="scenario",
    arangodb_hosts="http://localhost:8529",
    arangodb_auth=dict(password="tiro-password"),
).bind(influxdb_bucket="tiro", influxdb_measurement="tiro_telemetry")

In [3]:
context = Context()
context.add_pump("tiro", pump)

In [4]:
context.bind(start=-timedelta(hours=1), step=timedelta(minutes=30))

In [14]:
print(
    context.tiro_table(
        "%%Rack%%FlowRate", 
        type="historian",
        column="Rack",
        asset_agg_fn="sum",
    )["FlowRate"].value
)

rack_0   rack_1
2022-09-28 12:30:00+08:00     0.00     0.00
2022-09-28 13:00:00+08:00  2150.16  2649.47
2022-09-28 13:30:00+08:00  1922.42  1848.53

In [27]:
print(
context.tiro_table(
    "%%ActivePower",
    type="status",
    as_dataframe=False,
    value_only=True,
).value
)

{
    'DataHall': {
        'data_hall_0': {
            'CRAC': {'crac_0': {'Telemetry': {'ActivePower': 469.97}}},
            'Rack': {
                'rack_0': {'Telemetry': {'ActivePower': 926.17}},
                'rack_1': {'Telemetry': {'ActivePower': 924.3}}
            }
        }
    }
}

In [35]:
print(
context.tiro_table(
    "%%FlowRate",
    type="status",
    as_dataframe=True,
    include_tags=["Rack", "Server"],
    value_only=True
)["FlowRate"].value
)

Rack    Server   value
0  rack_0  server_0  527.90
1  rack_0  server_1  597.79
2  rack_0  server_2  587.88
3  rack_0  server_3  863.72
4  rack_1  server_3  863.72
5  rack_1  server_4  590.90
6  rack_1  server_5  865.13
7  rack_1  server_6  357.40
8  rack_1  server_7  125.10

In [9]:
s1 = t1["ActivePower"].value
s1

,path,DataHall,CRAC,type,value,Rack,Server
asset_path,,,,,,,
DataHall.data_hall_0.CRAC.crac_0,DataHall.CRAC.ActivePower,data_hall_0,crac_0,Telemetry,729.86,NaN,NaN
DataHall.data_hall_0.Rack.rack_0,DataHall.Rack.ActivePower,data_hall_0,NaN,Telemetry,576.61,rack_0,NaN
DataHall.data_hall_0.Rack.rack_1,DataHall.Rack.ActivePower,data_hall_0,NaN,Telemetry,808.42,rack_1,NaN


In [54]:
query="""
DataHall:
  Rack:
    $name_match: _1
    Server:
      FlowRate:
        $lt: 400
  CRAC:
    ActivePower:
    _ReturnTemperature:
      $lt: 15
      $gt: 25
"""
query_dict = yaml.safe_load(query)

In [55]:
print(
context.tiro_table(
    yaml.safe_load(query),
    type="status",
    as_dataframe=False,
    value_only=True
).value
)

{
    'DataHall': {
        'data_hall_0': {
            'Rack': {
                'rack_1': {
                    'Server': {
                        'server_4': {'Telemetry': {'FlowRate': 463.1}},
                        'server_5': {'Telemetry': {'FlowRate': 530.32}},
                        'server_6': {'Telemetry': {'FlowRate': 686.7}},
                        'server_7': {'Telemetry': {'FlowRate': 451.27}}
                    }
                }
            }
        }
    }
}